In [1]:
!pip install pyspark gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()  # fake_news_model.zip dosyasını yükle


Saving fake_news_model.zip to fake_news_model.zip


In [4]:
import zipfile

with zipfile.ZipFile("fake_news_model.zip", 'r') as zip_ref:
    zip_ref.extractall("./")


In [6]:
!ls -R fake_news_model


fake_news_model:
ml_model  preprocessing_pipeline

fake_news_model/ml_model:
data  metadata

fake_news_model/ml_model/data:
part-00000-1fdc2f49-5e3e-4a87-aec2-a105ca894ca5-c000.snappy.parquet  _SUCCESS

fake_news_model/ml_model/metadata:
part-00000  _SUCCESS

fake_news_model/preprocessing_pipeline:
metadata  stages

fake_news_model/preprocessing_pipeline/metadata:
part-00000  _SUCCESS

fake_news_model/preprocessing_pipeline/stages:
0_Tokenizer_ea73d0c7c743	 2_HashingTF_4fb752621ebf
1_StopWordsRemover_9e616728d4ca  3_IDF_eefdce583c81

fake_news_model/preprocessing_pipeline/stages/0_Tokenizer_ea73d0c7c743:
metadata

fake_news_model/preprocessing_pipeline/stages/0_Tokenizer_ea73d0c7c743/metadata:
part-00000  _SUCCESS

fake_news_model/preprocessing_pipeline/stages/1_StopWordsRemover_9e616728d4ca:
metadata

fake_news_model/preprocessing_pipeline/stages/1_StopWordsRemover_9e616728d4ca/metadata:
part-00000  _SUCCESS

fake_news_model/preprocessing_pipeline/stages/2_HashingTF_4fb752621ebf:
meta

In [10]:
import gradio as gr
from pyspark.sql import SparkSession, Row
from pyspark.ml import PipelineModel
import os
from pyspark.ml.classification import NaiveBayesModel

# Spark başlatıcı
def initialize_spark(app_name="FakeNewsApp"):
    return SparkSession.builder \
        .appName(app_name) \
        .master("local[*]") \
        .getOrCreate()

# Modeli yükleme

def load_complete_pipeline(spark, model_path):
    preprocessing_model = PipelineModel.load(os.path.join(model_path, "preprocessing_pipeline"))
    model = NaiveBayesModel.load(os.path.join(model_path, "ml_model"))
    model_type = "Naive Bayes"
    return preprocessing_model, model, model_type

# Tekli haber tahmin fonksiyonu
def predict_single_text(spark, preprocessing_model, model, text):
    df = spark.createDataFrame([Row(text=text)])
    transformed = preprocessing_model.transform(df)
    prediction = model.transform(transformed).select("prediction").collect()[0][0]
    return prediction

# Gradio tahmin fonksiyonu
def gradio_predict(text):
    spark = initialize_spark()
    try:
        preprocessing_model, model, model_type = load_complete_pipeline(spark, "./fake_news_model")
        prediction = predict_single_text(spark, preprocessing_model, model, text)
        result = "Gerçek Haber ✅" if prediction == 1.0 else "Yalan Haber ❌"
        return f"{result} (Model: {model_type})"
    except Exception as e:
        return f"Hata oluştu: {e}"
    finally:
        spark.stop()

# Gradio arayüzü
iface = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Textbox(lines=5, placeholder="Haber metnini buraya yazınız..."),
    outputs=gr.Textbox(label="Tahmin Sonucu"),
    title="📰 Yalan Haber Tespit Sistemi",
    description="Bu arayüz, haber metninizin doğruluğunu tahmin eder.",
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a34cff060777d9e888.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
